### Import data

In [1]:
import pandas as pd
df = pd.read_csv('./result/1by1+predictions.csv')
print(df.columns)

Index(['topic', 'stance', 'argument', 'key_point'], dtype='object')


In [2]:
print(df)

                                              topic  stance  \
0     Assisted suicide should be a criminal offence       1   
1     Assisted suicide should be a criminal offence       1   
2     Assisted suicide should be a criminal offence       1   
3     Assisted suicide should be a criminal offence       1   
4     Assisted suicide should be a criminal offence       1   
...                                             ...     ...   
7233           The USA is a good country to live in       0   
7234           The USA is a good country to live in       0   
7235           The USA is a good country to live in       0   
7236           The USA is a good country to live in       0   
7237           The USA is a good country to live in       0   

                                               argument  key_point  
0     a cure or treatment may be discovered shortly ...        NaN  
1     allow assisted suicide could allow unscrupulou...        NaN  
2     Allowing assisted suicide woul

### Generate KPs

In [12]:
import openai
import yaml
with open("conf/index.yaml") as f:
    credentials = yaml.safe_load(f)
openai.api_key = credentials['environment_variables']['OPENAI_API_KEY']
model_name = "gpt-4"

In [16]:
def generate_kp(topic, argument):
    system_prompt = f"""
        You need to do key point analysis on the query of user and
        generate a key point from it based on the topic "{topic}".
        The return format is as followed: Key Point: ...
        """

    rep = openai.ChatCompletion.create(
        model=model_name,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": argument}
        ]
    )
    result = rep["choices"][0]["message"]["content"]
    try:
        key_point = result.split("Key Point: ")[1].strip()
        # print(result)
        return key_point
    except (IndexError, KeyError)as e:
        print(f"Error: {e}. Regenerating string...")
        return generate_kp(topic, argument)


In [26]:
for index,row in df.iloc[7000:].iterrows():
    kp = generate_kp(row['topic'],row['argument'])
    print(index)
    print(kp)
    df.at[index, 'key_point'] = kp

df.to_csv('./result/1by1+predictions.csv', index=False)
print("Add new data successfully!!!!!")

7000
Social media platforms should remain private and free from government intervention.
7001
Regulation of social media by the government could lead to censorship and affect freedom of expression.
7002
Concerns over privacy and personal information on social media, given governmental regulation.
7003
Social media platforms should maintain freedom and not be controlled by any government.
7004
Regulation of social media platforms by the government can infringe on citizens' freedom of expression and privacy.
7005
Government regulation of social media platforms can potentially infringe on individuals' freedom of expression.
7006
Opposition to government regulation of social media based on freedom of speech rights, except in cases of clear law violation.
7007
Government regulation of social media platforms can infringe on freedom of expression and thought.
7008
People should have the freedom to post their ideas and government regulation of social media would infringe on this right.
7009
En

### Check chinese characters

In [1]:
import pandas as pd
df = pd.read_csv('./result/1by1+predictions.csv')
print(df.columns)

Index(['topic', 'stance', 'argument', 'key_point', 'cos_sim', 'matched_kp',
       'embedding(0_1_2_3)', 'embedding(4_7_8_15)', 'embedding(5_6_9_10)',
       'embedding(11_12_13_14)', 'embedding(16_17_18_19)',
       'embedding(20_21_22_23)', 'embedding(24_25_26_27)', 'avg_embedding'],
      dtype='object')


In [2]:
import re
def contains_chinese(text):
    # Regular expression pattern for Chinese characters
    chinese_pattern = re.compile('[\u4e00-\u9fff]+')

    # Search for Chinese characters in the text
    if chinese_pattern.search(text):
        return True
    else:
        return False

In [3]:
for index, row in df.iterrows():
    # print(str(index)+": ")
    if contains_chinese(df.at[index,'key_point']):
        print(str(index)+": ")
        # print("old: "+df.at[index,'key_point'])
        # topic = row['topic']
        # argument = row['argument']
        # result = generating(topic, argument)
        # df.at[index, 'key_point'] = result
    # else: print("No")